In [1]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
from enum import Enum

In [2]:
OSM_TAGS = {'name': True, # street name
            
            'oneway': True, # one way + dne

            'turn:lanes': True, # arrow markings
            'turn:lanes:forward': True, # arrow markings
            'turn:lanes:backward': True, # arrow markings

            'lanes': True, # lane count
            'lanes:forward': True, # lane count
            'lanes:backward': True, # lane count 

            'maxspeed': True, # speed limit
            'maxspeed:forward': True, # speed limit
            'maxspeed:backward': True, # speed limit
            }

MAP_FEATURES = list(OSM_TAGS.keys())

class MapFeatures(Enum):
    STREET_NAME = "name"
    ONEWAY = "oneway"
    TURN_LANES = "turn:lanes"
    TURN_LANES_FWD = "turn:lanes:forward"
    TURN_LANES_BWD = "turn:lanes:backward"
    LANES = "lanes"
    LANES_FWD = "lanes:forward"
    LANES_BWD = "lanes:backward"
    MAXSPEED = "maxspeed"
    MAXSPEED_FWD = "maxspeed:forward"
    MAXSPEED_BWD = "maxspeed:backward"

def load_metadata(file_path):
    metadata = pd.read_csv(file_path)
    metadata['geometry'] = metadata['geometry'].apply(loads)
    return metadata

def preprocess(string):
    string = string.lower()
    string = string.split(' ')
    string = [s.strip().replace('.', '').replace(',', '') for s in string]
    string = ' '.join(string)

def equals(gt_value, pred_value, map_feature):
    if map_feature == 'name':
        return preprocess(gt_value) == preprocess(pred_value)
    else:
        return gt_value == pred_value

def get_pred_status(gt_value, pred_value, map_feature):
    if pd.isna(gt_value) and pd.isna(pred_value):
        return 'tn'
    elif not pd.isna(gt_value) and pd.isna(pred_value):
        return 'fn'
    elif pd.isna(gt_value) and not pd.isna(pred_value):
        return 'fp'
    # treat a mismatch as a fn
    elif equals(gt_value, pred_value, map_feature):
        return 'tp'
    else:
        return 'fn'

def plot_pred_map(gdf):
    return gdf.explore(tiles="cartodb positron", 
                       color=gdf['color'],
                       popup=True,
                       style_kwds={'weight': 7, 'opacity': 0.5})
                       
def eval_map_feature_pred(gt_gdf, pred_df, map_feature):
    map_feature = map_feature.value
    assert map_feature in MAP_FEATURES, f"Invalid map feature: {map_feature}"

    map_feature_gt_gdf = gt_gdf[['osmid', 'geometry', map_feature]]
    map_feature_pred_df = pred_df[['osmid', map_feature]]
    map_feature_gt_gdf = map_feature_gt_gdf.merge(map_feature_pred_df, on='osmid', suffixes=('_gt', '_pred'))

    map_feature_gt_gdf['pred_status'] = map_feature_gt_gdf.apply(lambda x: get_pred_status(x[map_feature + '_gt'], 
                                                                                           x[map_feature + '_pred'], 
                                                                                           map_feature), axis=1)

    map_feature_gt_gdf = map_feature_gt_gdf[map_feature_gt_gdf['pred_status'].isin(['tp', 'fp', 'fn'])]
    tp_samples = map_feature_gt_gdf[map_feature_gt_gdf['pred_status'] == 'tp']
    fp_samples = map_feature_gt_gdf[map_feature_gt_gdf['pred_status'] == 'fp']
    fn_samples = map_feature_gt_gdf[map_feature_gt_gdf['pred_status'] == 'fn']

    map_feature_gt_gdf['color'] = map_feature_gt_gdf['pred_status'].map({'tp': 'green', 
                                                                         'fp': 'red', 
                                                                         'fn': 'orange'})

    precision = len(tp_samples) / (len(tp_samples) + len(fp_samples)) if (len(tp_samples) + len(fp_samples)) > 0 else 0
    recall = len(tp_samples) / (len(tp_samples) + len(fn_samples)) if (len(tp_samples) + len(fn_samples)) > 0 else 0
    print(f"Precision for map feature {map_feature} is {precision*100}%")
    print(f"Recall for map feature {map_feature} is {recall*100}%")

    return map_feature_gt_gdf

## Load the photos and way sections metadata

In [4]:
photos_df = load_metadata('../metadata/photos.csv')
way_sections = load_metadata('../metadata/ways.csv')

# process them to get predictions (e.g. demo_workflow.ipynb)
# we will use our existing LLM predictions

## Load the ground truth data

In [5]:
gt_csv = pd.read_csv('../metadata/ground_truth.csv')
gt_gdf = gpd.GeoDataFrame(gt_csv.merge(way_sections, on='osmid'), geometry='geometry', crs='EPSG:4326')

## Load your predictions

In [7]:
# pred_df = '...load your predictions dataframe'
pred_df = pd.read_csv('../metadata/claude_3.5sonnet_predictions.csv')

## Evaluate your predictions on a map feature

In [ ]:
pred_eval_df = eval_map_feature_pred(gt_gdf, pred_df, MapFeatures.LANES)

## Plot the results

In [ ]:
plot_pred_map(pred_eval_df)